In [1]:
using LinearAlgebra

# 1. Modified Gram-Schmidt

The Gram-Schmidt process performs poorly numerically: the resulting vectors are often not quite orthogonal due to rounding errors.

$\qquad$ An improved version that results in smaller errors in finite precision arithmetic removes the parallel components
as soon as they are available<br>
$\qquad$ we know $A = Q R:$ we cans solve for each $q_i$ and a row of $R$ at a time.  

## 1.1 The Vector $\mathbf{q_1}$ and the first Row of $\mathbf{R}$

We know $A = Q R$. Partitioning the $Q$ matrix into  $\left( \begin{array}{c|ccc} q_1 & q_2 & \dots q_n \end{array} \right)$
and $R = \left( \begin{array}{c|ccc} r_{1 1} & r_{1 2} & \dots & r_{1 n} \\ \hline
                                     0       & r_{2 2} & \dots & r_{2 n} \\
                                     0       & 0       & \dots & r_{3 n} \\
                                     \dots   & \dots   & \dots & \dots \\
                                     0       & 0       & \dots & r_{n n}
                                     \end{array}\right)$, we obtain

$\qquad\begin{align}
a_1 &= r_{1 1}\ q_1                & \Leftrightarrow \quad & r_{1 1}\ q_1 = a_1 \\
a_2 &= r_{1 2}\ q_1 + r_{2 2}\ q_2 & \Leftrightarrow \quad & r_{1 2}\ q_1 = a_2 - r_{2 2}\ q_2 \\
\dots& & &\\
a_n &= r_{1 n}\ q_1 + r_{2 n}\ q_2 + \dots r_{n n}\ q_n \quad& \Leftrightarrow \quad & r_{1 n}\ q_1  = a_n - + r_{2 n}\ q_2 \dots - r_{n n}\ q_n
\end{align}$

Which we can solve for the $q_1$ and the first row of the $R$ matrix by taking dot products with the $q_1$ vector:

$\qquad\begin{align}
r_{1 1} &= \Vert a_1 \Vert,\quad q_1 = \frac{1}{r_{1 1}} a_1 \\
r_{1 i} &= a_i \cdot q_1,\quad i=2,3,\dots n
\end{align}$

since the $q_i$ are orthonormal.

## 1.2 Each of the Remaining Vectors $\mathbf{q_i}$ and Corresponding Rows of $\mathbf{R}$

The remaining equations are $A_2 = Q_2 R_2,$<br>
$\qquad$ where $A_2 = \left( a_2\;\dots\;a_n \right)- q_1 \left( r_{1 2} \; \dots r_{1 n} \right),\quad Q_2 = \left( q_2 \dots q_n \right) \;\;$ and
$\;\;R_2 = \left( \begin{array}{ccc} r_{2 2} & \dots & r_{2 n} \\
                                     0       & \dots & r_{3 n} \\
                                     \dots   & \dots & \dots \\
                                     0       & \dots & r_{n n}
                                     \end{array}\right)$

Since $R_2$ is upper triangular, this new problem is solved as before,<br>
$\qquad$ with each of the columns in $A_2$ given by
$a^{(1)}_i = a_i - r_{1 i} q_1, \quad i=2,3,\dots n$.

$\qquad\begin{align}
r_{2 2} &= \Vert a^{(1)}_2 \Vert, \quad q_2 = \frac{1}{r_{2 2}} a^{(1)}_2 \\
r_{2 i} &= a^{(1)}_i \cdot q_2, \quad i=3, \dots n
\end{align}$

The result is a reduced problem that again has the same form.

The modified Gram-Schmidt procedure computes the $R$ matrix one row at a time.<br>
Below a reference implementation:

In [4]:
function modified_gram_schmidt(A)
    # orthogonalises the columns of the input matrix A, updating the vectors in a copy of A: A -> Q
    num_vectors = size(A)[2]
    Q           = copy(A)

    for i = 1:num_vectors
        r_ii = norm(Q[:, i])
        Q[:, i] = Q[:, i] / r_ii               # compute the new q
        for j = (i+1) : num_vectors            # for each remaining vector
            r_ij     = dot(Q[:, j], Q[:, i])
            Q[:, j] -= r_ij * Q[:, i]          #     update remaing vectors in A
        end
    end
    return Q
end;

The Gram-Schmidt procedure by contrast computes the entries of $R$ one column at a time.

In [5]:
function classical_gram_schmidt(A)
    # orthogonalises the columns of the input matrix A, updating the vectors in a copy of A: A -> Q
    vec_size,num_vectors = size(A)
    Q                    = copy(A)

    for i = 1:num_vectors
        sum = zeros(vec_size)               # compute the orthogonal projection of a_i onto the current span
        for j = 1:(i-1)
            r_ji = dot(Q[:, j], Q[:, i])
            sum +=r_ji * Q[:, j]
        end
        Q[:, i] -= sum                      # compute the new w_i (the orthognal part of a_i)

        r_ii = norm(Q[:,i])
        Q[:, i] = Q[:, i] / r_ii            # normalize w_i to q_i
    end
    return Q
end;

In [8]:
A = [2. 6 5 9;
     1 2 1 -7;
     0 1 2 4;
     1 1 1 8
]
MQ=modified_gram_schmidt(A)
@show MQ'MQ ≈ I
MQ

MQ' * MQ ≈ I = true


4×4 Matrix{Float64}:
 0.816497   0.471405  -0.09245   0.320256
 0.408248  -0.235702  -0.3698   -0.800641
 0.0        0.471405   0.7396   -0.480384
 0.408248  -0.707107   0.5547    0.160128

In [9]:
CQ = classical_gram_schmidt(A)
@show MQ ≈ CQ
CQ

MQ ≈ CQ = true


4×4 Matrix{Float64}:
 0.816497   0.471405  -0.09245   0.320256
 0.408248  -0.235702  -0.3698   -0.800641
 0.0        0.471405   0.7396   -0.480384
 0.408248  -0.707107   0.5547    0.160128

# 2. Explore

Try various matrices with different sizes and condition numbers.
* How orthogonal are the resulting matrices?
* Investigate the errors using the Householder Algorithm instead